In [28]:
from my_weapon import *
from tqdm import tqdm_notebook as tqdm
import matplotlib
from collections import Counter

In [29]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
nlp = spacy.load('es', disable=['parser', 'ner'])

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [30]:
import unicodedata

def normalize_lower(text):
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode().lower()

In [31]:
stop_words = json.load(open("data/spanish-stop-words.json"))["words"]
stop_words = [normalize_lower(w) for w in stop_words]
stop_words.extend([
    "rt", "…", "...", "URL", "http", "https", "“", "”", "‘", "’", "get", "2", "new", "one", "i'm", "make",
    "go", "good", "say", "says", "know", "day", "..", "take", "got", "1", "going", "4", "3", "two", "n",
    "like", "via", "u", "would", "still", "first", "that's", "look", "way", "last", "said", "let",
    "twitter", "ever", "always", "another", "many", "things", "may", "big", "come", "keep", "RT",
    "5", "time", "much", "_", "cound", "-", '"'
])
stop_words.extend([',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%', '|'])
stop_words = set(stop_words)

In [33]:
texts_out = []    
for line in tqdm(open("data/LDA_corpus.txt")):
    ## words = [w for w in line.strip().split() if w not in stop_words and len(w) > 1]
    words = [w for w in line.strip().split() if w not in stop_words]
    texts_out.append(words)

#         # Create Dictionary
#         self.id2word = corpora.Dictionary(texts_out)

#         # Create Corpus
#         self.texts = texts_out

#         # Term Document Frequency
#         self.corpus = [self.id2word.doc2bow(text) for text in texts_out]

In [15]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
# nlp = spacy.load('en')
import re

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    i = 0
    for sent in tqdm(texts):
        sent = " ".join(sent)
        sent = re.sub(r'#(\w+)', r'itstopiczzz\1', sent)
        sent = re.sub(r'@(\w+)', r'itsmentionzzz\1', sent)
        doc = nlp(sent)
        
        _d = [token.lemma_ for token in doc if token.pos_ in allowed_postags and token.lemma_ not in stop_words and token.lemma_]
#         _d = [(token.pos_, token.lemma_) for token in doc if token.lemma_ not in stop_words]
        
        _d = [x.replace('itstopiczzz', '#') for x in _d]
        _d = [x.replace('itsmentionzzz', '@') for x in _d]
        texts_out.append(_d)

    return texts_out

texts_out = lemmatization(texts_out)

In [34]:
# Create Dictionary
id2word = corpora.Dictionary(texts_out)

# Create Corpus
texts = texts_out

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)]]


In [35]:
# train new
from gensim.models import CoherenceModel, LdaModel

lda_model = LdaModel.load("data/LDA-5.mod")
# lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=7, chunksize=1000, random_state=43)

In [36]:
lda_model.print_topics()

[(0,
  '0.079*"lopez-gatell" + 0.028*"pandemia" + 0.024*"cubrebocas" + 0.023*"dr" + 0.017*"salud" + 0.015*"@sinlineamx" + 0.015*"vacuna" + 0.013*"carta" + 0.012*"obrador" + 0.012*"@lillytellez"'),
 (1,
  '0.043*"hugo" + 0.018*"te" + 0.012*"tu" + 0.012*"mexicanos" + 0.010*"cul" + 0.010*"@brozoxmiswebs" + 0.010*"peda" + 0.010*"envie" + 0.008*"@nachorgz" + 0.007*"cientifico"'),
 (2,
  '0.172*"lopez" + 0.081*"mexico" + 0.048*"gobernadores" + 0.037*"covid" + 0.033*"19" + 0.026*"@rochaperiodista" + 0.021*"COVID" + 0.019*"fernandez" + 0.013*"personas" + 0.011*"hora"'),
 (3,
  '0.038*"presidente" + 0.019*"dias" + 0.016*"comercial" + 0.016*"casa" + 0.016*"@lopezobrador_" + 0.015*"q" + 0.013*"peores" + 0.013*"marzo" + 0.011*"estrategia" + 0.011*"|"'),
 (4,
  '0.191*"gatell" + 0.025*"renuncia" + 0.023*"mil" + 0.022*"muertos" + 0.021*"culpa" + 0.020*"A" + 0.019*"Y" + 0.015*"muertes" + 0.014*"DE" + 0.013*"piden"'),
 (5,
  '0.030*"fiesta" + 0.028*"super" + 0.024*"frente" + 0.020*"p" + 0.020*"ah" + 0

In [ ]:
# Compute Perplexity
print('Perplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [37]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [38]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.034426 -0.021653       1        1  22.261261
0     -0.302890 -0.295609       2        1  21.858269
4      0.353079 -0.250319       3        1  14.653995
3     -0.105505  0.125777       4        1  13.080056
2      0.047677  0.114471       5        1   9.937415
5      0.055814  0.149689       6        1   9.514930
6     -0.013749  0.177644       7        1   8.694070, topic_info=                 Term          Freq         Total Category  logprob  loglift
618            gatell  2.410458e+06  2.410458e+06  Default  30.0000  30.0000
215             lopez  1.469427e+06  1.469427e+06  Default  29.0000  29.0000
1705     lopez-gatell  1.483502e+06  1.483502e+06  Default  28.0000  28.0000
11             mexico  6.932380e+05  6.932380e+05  Default  27.0000  27.0000
1518             hugo  8.298330e+05  8.298330e+05  Default  26.0000  26.0000
...               ...           ...           ...      ...      ...      ...
751          nacional  4.954482e+04  4.954575e+04   Topic7  -5.0164   2.4425
3655         diciendo  4.783643e+04  4.783736e+04   Topic7  -5.0515   2.4425
14945   subsecretaria  4.805980e+04  4.806073e+04   Topic7  -5.0468   2.4425
298             venta  4.749376e+04  4.749469e+04   Topic7  -5.0587   2.4425
163919        andrade  8.976691e+04  8.977283e+04   Topic7  -4.4221   2.4425

[260 rows x 6 columns], token_table=         Topic      Freq          Term
term                                  
49           5  0.999962      #COVID19
1040059      2  0.000975  #sinpalabras
1040059      3  0.015712  #sinpalabras
1040059      5  0.011405  #sinpalabras
1040059      6  0.971896  #sinpalabras
...        ...       ...           ...
6566         1  0.999994         vayan
1232         1  0.999989           ven
298          7  0.999986         venta
1852         1  0.999985           veo
34           4  0.999990             |

[280 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 5, 4, 3, 6, 7])

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in tqdm(range(start, limit, step)):
        # model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
optimal_model = model_list[3]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

In [ ]:
lda_rst = {}

for line in open("data/lda_rst.txt"):
    '''
    Num Topics = 20  has Coherence Value of 0.3005
    '''
    if not line.startswith("Num"):
        continue
    w = line.strip().split()
    N = int(w[3]); score = float(w[-1])
    if N not in lda_rst:
        lda_rst[N] = []
    lda_rst[N].append(score)
# print(lda_rst)

In [ ]:
new_lda_rst = []
for N, scores in lda_rst.items():
    for s in scores:
        new_lda_rst.append({"N": N, "Coherence": s})

In [ ]:
new_lda_rst = pd.DataFrame(new_lda_rst)

In [ ]:
plt.figure(figsize=(9, 6))
sns.lineplot(x="N", y="Coherence", data=new_lda_rst)

In [ ]:
# After choosing the best model

lda_cohen = []

for line in open("lda_loglog.txt"):
    if line.startswith("Coherence Score: "):
        lda_cohen.append(float(line.strip().split()[-1]))

In [ ]:
max(lda_cohen)

In [ ]:
from gensim.models import CoherenceModel, LdaModel

lda_model = LdaModel.load("model/lda-ira-78.mod") # best model!